In [2]:
path = 'REC NOT CLOSED VFinale2.xlsm'
import pandas as pd
from jupyter_dash import JupyterDash
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
from dash import dcc
import numpy as np
import dash_bootstrap_components as dbc

In [3]:
def preprocess_data(path = '../RESOURCE SUMMARY_20220912.xlsx'):
    wb = pd.read_excel(path,'Sheet1',header=1)

    correspondences = {}

    nonworking = ['10th Day','9th Day','8th day','All Saints Day','Annual Leave','Ascension Day','Easter Monday','National Day',
    'Other Leave','Sick Leave','Special Leave granted by the DG','Whit Monday']

    non_audit = ['Administrative Matters & Support','Covid-19 Sanitary Situation','GC and EXB (incl. Annual report)',
        'HR Management & Recruitment', 'IOS Management - Ad-hoc Requests','IOS Team Meetings', 'JIU Coordination',
        'OAC support, preparation and meetings','Participation to UNESCO Working Groups or Task Forces',
        'Policy or Administrative Manual Item Review','Support to Investigation Unit',
        'Trainings or Workshops']

    audit = ['Audit-Ad-hoc request / Advisory','Audit-Annual Planning', 'Audit-QAIP (incl. TeamMate+ Migration)']

    reco = ['Audit-Recommendation Follow-up']

    correspondences['Nonworking'] = nonworking
    correspondences['Non audit time'] = non_audit
    correspondences['Audit Time'] = audit
    correspondences['Reco Follow-up Time'] = reco

    # Insert correspondences for the different Time Categories
    wb['Reporting categories'] = 'Audit Time'
    for i in correspondences: 
        wb['Reporting categories'].mask(wb['Time Category'].isin(correspondences[i]),i,inplace=True)

    # Make Phase 5 -- Recommendation followups be repored as Reco Follow up time
    wb['Reporting categories'].mask(wb['Phase'].astype('string').str.startswith('5'),'Reco Follow-up Time',inplace=True)

    # Remove IOS/... prefix from Assignment name
    prefix_removed = wb['Assignment Name'].str.extract(r'(^IOS/[a-zA-Z0-9_\./]+)-([a-zA-Z0-9_\' -]+)')[1]
    idxs = ~wb['Assignment Name'].str.startswith('IOS/').fillna(False)
    prefix_removed[idxs] = wb[idxs]['Assignment Name']
    wb['Assignment Name'] = prefix_removed


    return wb


In [4]:
wb = preprocess_data()

In [5]:
names = wb['Resource'].dropna().unique()

In [6]:
def options(names):
    options = []
    for i in names:
        d = {}
        d["label"] = i
        d["value"] = i
        options.append(d)
        
    return options

In [7]:
def drpdwn(i):
    opts = options(names)
    
    return dcc.Dropdown(options = opts, value = names,id = i,multi = True,className='dropdown')

In [8]:
hovertemplate = 'label = %{label}<br>Activity = %{customdata[0]}<br>Working hours = %{value}<extra></extra>'
colour_map = dict(zip(df['time'].sort_values().unique(),px.colors.carto.Safe))
colour_map['(?)'] = 'lightgrey'

app = JupyterDash(__name__,external_stylesheets=[dbc.themes.COSMO])

server = app.server

app.layout = html.Div(children = [html.Div(
			children= [html.H1('Time Utilization per Month and Auditor',className='heading'),
					html.Div(drpdwn('names'),className='dropdown'),
					dcc.Graph(figure = fig, id='graph', className='figure')
				]
			),
			dcc.Store(id='clientside-data',data=df)
		]
	)

@app.callback(
	Output('graph','figure'),
	Input('names','value'))
def update_figure(name):

	to_plot = df[df['Resource'].isin(name)]

	fig = px.treemap(to_plot,path=[px.Constant('Month'),'year_month','Resource','time','time_stat','activity','activity_hours'],
	hover_data=['time','activity_hours'],values='working_hours',color = 'time',color_discrete_map=colour_map,maxdepth=5,
	width = 1300,height=800)

	# fig.update_layout(title='Time Utilization per Month and Auditor',title_x=0.5)
	fig.update_traces(hovertemplate=hovertemplate)

	return fig
	
app.run_server()

KeyError: 'time'